In [2]:
# from pyspark import SparkContext
from pyspark.sql import SparkSession


TODO:
- To a 'describe' on the dataframe to describe the schema
- Give a description analysis of each column value
- Define the schema type for the dataframe

In [2]:
# sc=SparkContext.getOrCreate()
spark = SparkSession.builder.appName("EDA Vehicle").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/07 15:22:48 WARN Utils: Your hostname, DevanshNZ resolves to a loopback address: 127.0.1.1; using 172.17.62.21 instead (on interface eth0)
24/06/07 15:22:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/07 15:22:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
vehicle_spec_path = "../local_datasets/aaa_data/vehicle_spec_info.csv"

# Load vehicle spec dataframe:
df = spark.read.csv(vehicle_spec_path, header=True, inferSchema=True)

In [6]:
df.show()

+-------------------+------------+--------------------+--------------------------+----------+------------------------------+-------------------+-------------------+
|              Model|Vehicle Type|     Variant Details|Fast Charge Time (Minutes)|Range (km)|Energy Consumption (kWh/100km)|ANCAP Safety Rating|Listed Price ($AUD)|
+-------------------+------------+--------------------+--------------------------+----------+------------------------------+-------------------+-------------------+
|        Audi e-tron|   Large SUV|2022 Audi e-tron ...|      85 mins (5%-80% c...|       459|                          23.2|    [object Object]|               NULL|
|     Audi e-tron GT|   Large Car|2024 Audi e-tron ...|      23 mins (5%-80% c...|       540|                          19.2|    [object Object]|             182400|
|   BMW 5 Series BEV|   Large Car|2024 BMW i5 eDriv...|      30 mins (10%-80% ...|       550|                          16.5|    [object Object]|             155900|
|         

- Looking initially, there is missing info for fast charge time.
    - Not only is there missing info, but the charge ranges are different, need to find way to normalise.
- The ANCAP Safety Rating was not imported, might have to remove or web scrape and update.
- Soem Listed prices are missing from the table. Will need to fill the NULL values.

In [14]:
# ** Replace NULL values in price column **
"""
NULL values will be filled with prices found on carsales.com.au 07/06/2024
"""

null_variants = df.filter( df['Listed Price ($AUD)'].isNull() ).select("Variant Details")
null_variants = null_variants.rdd.flatMap(lambda x: x).collect()
null_variants

[Row(Variant Details='2022 Audi e-tron 55 Auto quattro MY23'),
 Row(Variant Details='2023 Genesis G80 Auto AWD MY23'),
 Row(Variant Details='2023 Hyundai Kona Electric Elite Auto MY23'),
 Row(Variant Details='2023 Mercedes-Benz EQC EQC400 Auto 4MATIC')]

In [11]:
# Prices found:
"""
2022 Audi e-tron 55 Auto quattro MY23  |  $119,999
2023 Genesis G80 Auto AWD MY23  |  $74,690
2023 Hyundai Kona Electric Elite Auto MY23  |  $54,500
2023 Mercedes-Benz EQC EQC400 Auto 4MATIC  |  $92,831
"""

NoneType